# Cross Validation Index Generation

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
import pickle
import joblib
import os

## Load data

In [2]:
df_config = pd.read_csv('../data/config.csv')
train_id = 5
df_config

,hash_id,label,wav_files_path,processed_file_folder,processed_file_path,wav_files_info,cv_alg,cv_folds,cv_path,preproc_alg,...,target_id_file,model_path,model_inits,model_neurons,model_status,model_epochs,model_patience,model_learning_rate,model_optimizer,model_batch_size
0,3786470895109500580,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/3786470895109500580_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,5,../data/models/3786470895109500580_hidden_neur...,../data/models/3786470895109500580_model_statu...,1000,100,0.001,adam,NaN
1,-5662987709573759986,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/-5662987709573759986_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,5,../data/models/-5662987709573759986_hidden_neu...,../data/models/-5662987709573759986_model_stat...,1000,100,0.001,adam,NaN
2,3837049506038808913,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/3837049506038808913_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,1,../data/models/3837049506038808913_hidden_neur...,../data/models/3837049506038808913_model_statu...,100,10,0.001,adam,NaN
3,4644399470053765538,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/4644399470053765538_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,1,../data/models/4644399470053765538_hidden_neur...,../data/models/4644399470053765538_model_statu...,20,10,0.001,adam,NaN
4,5754269668442876343,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/5754269668442876343_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,2,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,2,../data/models/5754269668442876343_hidden_neur...,../data/models/5754269668442876343_model_statu...,10,5,0.001,adam,100.0
5,-8564343657574404315,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/-8564343657574404315_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/models/train_id_file.csv,../data/models,2,../data/models/-8564343657574404315_hidden_neu...,../data/models/-8564343657574404315_model_stat...,100,5,0.001,adam,100.0


### Load train data

In [3]:
df_train = pd.read_csv(df_config['train_data_path'][train_id])

In [4]:
df_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,8.903746,3.184031,0.052501,-9.532997,1.425378,10.894728,0.170885,-0.774334,-10.986876,11.461837,...,10.039516,-7.124313,7.238684,1.278997,11.078094,-10.535044,-0.140006,-8.772590,1.981356,1
1,9.981823,-20.750982,-0.942298,8.656325,0.158986,-10.275792,-0.552505,-0.059739,-11.232473,9.416682,...,-9.691469,18.482490,-11.253873,0.430396,9.836899,10.421699,1.471094,-10.779757,-0.711336,0
2,9.547097,2.984926,0.968209,-10.001620,-0.022503,9.344954,-0.471234,-0.959938,-10.124597,9.963728,...,10.926023,-3.391440,10.230726,1.108343,15.102184,-8.711126,-2.805098,-11.888262,0.759911,1
3,8.357513,20.604116,-0.611071,9.573068,2.036371,9.170248,0.270779,-0.285000,9.534010,9.546351,...,6.925222,-14.570370,-12.142899,-1.077644,10.180091,-10.983669,0.476673,12.383629,-1.151954,1
4,11.301140,21.471281,-0.948023,7.494141,-2.700322,10.126694,-0.853863,-1.600656,10.207078,8.708084,...,8.477780,-11.371481,-9.151545,0.073678,12.375645,-9.216089,-1.500698,10.151696,-0.249882,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,12.914528,-21.858322,-0.082550,8.630998,-0.867227,-9.497436,0.545329,-0.176121,-12.850332,10.312472,...,-9.239520,16.888234,-10.230283,0.649909,9.753831,10.021164,0.577720,-8.106964,-0.657299,0
99996,9.312289,-19.650764,0.069646,9.550809,0.845729,-10.820493,1.461406,0.434090,-9.811045,10.602906,...,10.527503,3.968774,8.401132,-0.156847,-11.755451,8.854734,2.588104,-10.794043,-1.202647,3
99997,-9.172817,0.358024,-1.123573,11.744890,-0.324210,-9.044175,-0.491234,0.873331,-11.669646,10.182295,...,8.258175,-21.258283,11.081116,-0.884730,9.933218,-10.225116,-1.108391,8.611485,0.443692,2
99998,-8.006341,15.809853,0.172775,-5.979533,-0.890775,12.643761,1.396864,-1.061963,9.541398,-11.902583,...,9.858991,-7.224331,7.128343,0.750136,10.906366,-12.132554,2.893459,-9.295337,-0.895020,0


# Stratified K-Folds

In [5]:
from sklearn.model_selection import StratifiedKFold
import pickle
import os

file_path = df_config['cv_path'][train_id]
n_folds = df_config['cv_folds'][train_id]

cv = StratifiedKFold(n_splits=n_folds)

data = df_train.drop(columns=['target']).values
target = df_train['target'].values

for idx, (trn_idx, val_idx) in enumerate(cv.split(data,target)):
    file_name = '%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                      idx, n_folds)
    with open(os.path.join(file_path,file_name),'wb') as file_handler:
        pickle.dump([trn_idx,val_idx],file_handler)

In [6]:
data.shape

(100000, 20)

In [7]:
target.shape

(100000,)